Create a LangChain agent that can:

Retrieve answers from documents (using vector DB or simple text).

Perform calculations (using a calculator tool).

Use AgentExecutor to decide which tool to use(based on description and user_input)

In [1]:
from langchain_openai import OpenAI,OpenAIEmbeddings
from langchain.agents import Tool,initialize_agent,load_tools
from langchain.agents.agent_types import AgentType
from langchain.tools import tool
from langchain.tools import DuckDuckGoSearchRun

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [14]:
loader = PyPDFLoader(r"C:\Users\Lenovo\Desktop\LANGCHAIN_PROJECT\1-Langchain\3.3-Data Trnasformer\attention.pdf")
docs = loader.load()
docs[0].page_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurren

In [18]:
splitter = CharacterTextSplitter(chunk_size=400,chunk_overlap=100)
documents = splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents,embeddings)
retriever = vectorstore.as_retriever()

In [33]:
# Define Tools(Retriever+Calculator)
@tool
# 3️⃣ Define the document search tool
def document_search(query: str) -> str:
    """Search uploaded document and return relevant content."""
    result = retriever.get_relevant_documents(query)
    if result:
        raw_text = result[0].page_content
        cleaned = " ".join(raw_text.split())  # removes \n and extra whitespace
        return cleaned
    else:
        return "No relevant info found."
        # return result[0].page_content if result else "No relevant info found."

document_tool = Tool(name="Document_Search",func=document_search,
                     description="answer the questions based on provided document")
# calculator
llm = OpenAI(temperature=0)
math_tools = load_tools(["llm-math"],llm=llm)

# combine tools
tools = [document_tool]+math_tools

# initialize agent
agent = initialize_agent(
    tools = tools,
    llm=llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 7️⃣ Ask agent a mixed question
question1 = "What is the main idea of the document?"
question2 = "What is 234 * 17 + 45?"

print("\nAnswer 1:")
print(agent.run(question1))

print("\nAnswer 2:")
print(agent.run(question2))

    


Answer 1:


> Entering new AgentExecutor chain...
 I should use the Document_Search tool to search for keywords or phrases that summarize the main idea.
Action: Document_Search
Action Input: 'main idea'
Observation: Input-Input Layer5 The Law will never be perfect , but its application should be just - this is what we are missing , in my opinion . <EOS> <pad> The Law will never be perfect , but its application should be just - this is what we are missing , in my opinion . <EOS> <pad> Input-Input Layer5 The Law will never be perfect , but its application should be just - this is what we are missing , in my opinion . <EOS> <pad> The Law will never be perfect , but its application should be just - this is what we are missing , in my opinion . <EOS> <pad> Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top: Full attentions for head 5. Bottom: Isolated attentions from just the word ‘its’ for attention heads 5 and 6. Note that the attentions 

# "Zero-Shot":
The agent has no training examples.

It understands tasks purely from the tool descriptions and the user query.
# "ReAct":
The agent performs reasoning (thinking step-by-step),

and acts (calls a tool) as needed.

# "DESCRIPTION":
Each tool you pass has a description (e.g., “Useful for math operations”).

The agent uses these descriptions to choose which tool to call.